GWAS summery stat data preprocess
1.need to convert Markername to rsID by combining Chr and basepair coordinate in _lifted bim file and and replace Markername by matching
2.calculate z-score using P-value and effect size using munge_sumstat in ldsc github repository

In [1]:
#read meta5 file -- this is our GWAS summery stat data
! zless /data/CARD/PD/summary_stats/META5_all.gz

MarkerName      Allele1 Allele2 Freq1   FreqSE  MinFreq MaxFreq Effect  StdErr  P-value Direction       HetISq  HetChiSq        HetDf   HetPVal freqSpan
chr11:88249377  t       c       0.9905  0.0026  0.9882  0.9971  0.0238  0.0594  0.6889  -+?+++--+???-+++?       20      13.753  11      0.247   0.00890000000000002
chr1:60320992   a       g       0.9375  0.0049  0.9221  0.9473  0.0185  0.021   0.3801  +-?--+?++??-+---+       20.1    15.022  12      0.2402  0.0252
chr8:135908647  a       g       0.2112  0.007   0.1703  0.2316  -0.0054 0.0123  0.6598  --?+++?--??-----+       27.2    16.492  12      0.1697  0.0613
chr16:77148858  a       g       0.9963  5e-04   0.9961  0.9984  -0.0228 0.1356  0.8663  -??-+??+-???----?       0       4.331   8       0.8261  0.00229999999999997
chr11:97895884  c       g       0.0587  0.0035  0.0361  0.0685  -0.0219 0.0217  0.3148  +-?--+?--??+---++       0.2     12.029  12      0.4433  0.0324
chr2:80117945   a       g       0.9979  7e-04   0.9968  0.9988  -0

In [ ]:
#copy meta 5 file into data folder - run only once for download
! cp -R /data/CARD/PD/summary_stats/META5_all.gz /data/songy4/TWAS/data/

#unzip META5_all.gz - run only once for download
! gzip -d META5_all.gz

#make sumstat.csv which contains MarkerName, Allele1, Allele2, Effect, StdErr, P-value
! awk '{print $1, $2, $3, $8, $9, $10}' data_folder/META5_all > data_folder/sumstat.txt

In [2]:
! awk '{print $1, $2, $3, $8, $9, $10}' data_folder/META5_all > data_folder/sumstat.txt

^C


In [1]:
#import library
import subprocess
import sys
import os
import shutil
import pandas as pd
import numpy as np

In [2]:
# set paths
basedir = '/data/songy4/twas'
datadir = f'{basedir}/data_folder'
fusiondir = f'{basedir}/fusion_twas'
geno_path = f'{datadir}/qc_genotypes_twas'
gene_list_path = f'{datadir}/gene_list.txt'
pheno_path = f'{datadir}/expression_matrix_final.txt'
coord_path = f'{datadir}/twas_coordinate.txt'
covar_path = f'{datadir}/covariates.txt'

In [3]:
#check sumstat.csv
meta = pd.read_csv(r"./data_folder/sumstat.txt", sep=' ')
print(meta.head())

#check the shape of meta
print("shape of meta data:", meta.shape)

#check the unique number of MarkerName in meta
print("Unique MarkerName in meta data:", meta['MarkerName'].nunique())

       MarkerName Allele1 Allele2  Effect  StdErr  P-value
0  chr11:88249377       t       c  0.0238  0.0594   0.6889
1   chr1:60320992       a       g  0.0185  0.0210   0.3801
2  chr8:135908647       a       g -0.0054  0.0123   0.6598
3  chr16:77148858       a       g -0.0228  0.1356   0.8663
4  chr11:97895884       c       g -0.0219  0.0217   0.3148
shape of meta data: (2721631, 6)
Unique MarkerName in meta data: 2721631


In [4]:
#open bim file 
bim_df = pd.read_csv(r"./data_folder/qc_genotypes_twas_hg19_lifted.bim", sep='\t', header=None,  names=('chr', 'rsid', 'kb', 'pos', 'a1', 'a2'))
bim_df.head()

,chr,rsid,kb,pos,a1,a2
0,1,rs145427775,0,10291,T,C
1,1,rs55998931,0,10492,T,C
2,1,rs199896944,0,13504,A,G
3,1,rs199856693,0,14933,A,G
4,1,rs201855936,0,14948,A,G


In [5]:
#combine chr and pos columns
bim_df['MarkerName'] = str('chr') + bim_df.chr.astype(str) + ':' + bim_df.pos.astype(str)

#check the shape of meta
print("shape of bim data:", bim_df.shape)

#check the shape of meta
print("number of unique MarkerName in bim data:", bim_df['MarkerName'].nunique())

bim_df.head()

shape of bim data: (23060234, 7)
number of unique MarkerName in bim data: 23060188


,chr,rsid,kb,pos,a1,a2,MarkerName
0,1,rs145427775,0,10291,T,C,chr1:10291
1,1,rs55998931,0,10492,T,C,chr1:10492
2,1,rs199896944,0,13504,A,G,chr1:13504
3,1,rs199856693,0,14933,A,G,chr1:14933
4,1,rs201855936,0,14948,A,G,chr1:14948


In [6]:
#add rsid column in meta by matching MarkerName from bim_df 
rename_dict = bim_df.set_index('MarkerName').to_dict()['rsid']
meta['SNP'] = meta['MarkerName'].map(rename_dict)

#count how many NaN in rsid in meta
print("number of NaN in rsid in meta data:", meta['SNP'].isna().sum())
print("number of rsid in meta data:", meta['SNP'].count())

meta.head()

number of NaN in rsid in meta data: 591918
number of rsid in meta data: 2129713


,MarkerName,Allele1,Allele2,Effect,StdErr,P-value,SNP
0,chr11:88249377,t,c,0.0238,0.0594,0.6889,rs11020170
1,chr1:60320992,a,g,0.0185,0.0210,0.3801,rs116406626
2,chr8:135908647,a,g,-0.0054,0.0123,0.6598,rs11992603
3,chr16:77148858,a,g,-0.0228,0.1356,0.8663,rs189372368
4,chr11:97895884,c,g,-0.0219,0.0217,0.3148,NaN


In [7]:
#drop rows in meta if rsid is NaN 
meta_df = meta[meta['SNP'].notna()]

#change Allele1 and Allele2 to A1 and A2
meta_df = meta_df.rename(columns={'Allele1':'A1', 'Allele2':'A2'})

#reset index
meta_df = meta_df.reset_index(drop=True)

#check numbers of rows in meta_df
print("numbers of rows in meta data:", meta_df.shape[0])

meta_df.head()

numbers of rows in meta data: 2129713


,MarkerName,A1,A2,Effect,StdErr,P-value,SNP
0,chr11:88249377,t,c,0.0238,0.0594,0.6889,rs11020170
1,chr1:60320992,a,g,0.0185,0.0210,0.3801,rs116406626
2,chr8:135908647,a,g,-0.0054,0.0123,0.6598,rs11992603
3,chr16:77148858,a,g,-0.0228,0.1356,0.8663,rs189372368
4,chr19:32978300,a,g,0.0398,0.0991,0.6879,rs62106378


In [8]:
#calculate z score with effect size and standard error

def z_score(a, b):
    if b == 0:
        return 0.0
    return float(a)/b

meta_df['Z'] = np.vectorize(z_score)(meta_df['Effect'], meta_df['StdErr'])

meta_df.head()

,MarkerName,A1,A2,Effect,StdErr,P-value,SNP,Z
0,chr11:88249377,t,c,0.0238,0.0594,0.6889,rs11020170,0.400673
1,chr1:60320992,a,g,0.0185,0.0210,0.3801,rs116406626,0.880952
2,chr8:135908647,a,g,-0.0054,0.0123,0.6598,rs11992603,-0.439024
3,chr16:77148858,a,g,-0.0228,0.1356,0.8663,rs189372368,-0.168142
4,chr19:32978300,a,g,0.0398,0.0991,0.6879,rs62106378,0.401615


In [9]:
#function to move columns
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

#rearrange Gene_Symbol, Chr, Start, Stop columns right nextg to index column 
sumstat = movecol(meta_df, cols_to_move=['SNP'], ref_col='MarkerName', place='After')

#drop MarkerName, Effect, StdErr, P-value colunms 
sumstat = sumstat.drop(['MarkerName', 'Effect', 'StdErr', 'P-value'], axis=1)

sumstat = sumstat.reset_index(drop=True)
print("shape of meta data:", sumstat.shape)
sumstat.head()

shape of meta data: (2129713, 4)


,SNP,A1,A2,Z
0,rs11020170,t,c,0.400673
1,rs116406626,a,g,0.880952
2,rs11992603,a,g,-0.439024
3,rs189372368,a,g,-0.168142
4,rs62106378,a,g,0.401615


In [10]:
#save meta_df as txt -- run only once for download
sumstat.to_csv(r'./data_folder/meta_1.txt', sep='\t' ,index=False)
#gwas sumary stat is ready to use

In [11]:
##grab significant GWAS hits (p <= 5e-8)
sig_meta = meta_df[meta_df['P-value']<= 5e-8]
#reset index
sig_meta = sig_meta.reset_index(drop=True)
print("shape of significant GWAS data:", sig_meta.shape)

sig_meta.head()

shape of significant GWAS data: (1381, 8)


,MarkerName,A1,A2,Effect,StdErr,P-value,SNP,Z
0,chr16:31054607,t,c,-0.0831,0.0099,6.178000e-17,rs1108431,-8.393939
1,chr10:121534732,t,c,0.0587,0.0095,5.506000e-10,rs196202,6.178947
2,chr5:102446462,a,c,-0.0603,0.0103,4.716000e-09,rs257320,-5.854369
3,chr16:31023273,a,g,-0.0852,0.0100,1.338000e-17,rs12447930,-8.520000
4,chr4:951040,t,g,0.1074,0.0097,1.054000e-28,rs3733344,11.072165


In [12]:
#add rsid column in meta by matching MarkerName from bim_df 
rename_dict = bim_df.set_index('rsid').to_dict()['pos']
sig_meta['pos'] = sig_meta['SNP'].map(rename_dict)
sig_meta.head()

,MarkerName,A1,A2,Effect,StdErr,P-value,SNP,Z,pos
0,chr16:31054607,t,c,-0.0831,0.0099,6.178000e-17,rs1108431,-8.393939,31054607
1,chr10:121534732,t,c,0.0587,0.0095,5.506000e-10,rs196202,6.178947,121534732
2,chr5:102446462,a,c,-0.0603,0.0103,4.716000e-09,rs257320,-5.854369,102446462
3,chr16:31023273,a,g,-0.0852,0.0100,1.338000e-17,rs12447930,-8.520000,31023273
4,chr4:951040,t,g,0.1074,0.0097,1.054000e-28,rs3733344,11.072165,951040


In [18]:
#save sig_meta -- run only once
sig_meta.to_csv(r'./data_folder/gwas_pos.csv', index=False)

In [13]:
#bring in coords
coords = pd.read_csv(coord_path, sep='\t')
coords.head()

,X.Chr,start,end,ID
0,1,69091,70008,ENSG00000186092
1,1,860260,879955,ENSG00000187634
2,1,879584,894689,ENSG00000188976
3,1,895967,901095,ENSG00000187961
4,1,901877,911245,ENSG00000187583


In [ ]:
#get ENSG ID from coords if pos in sig_meta are within start and stop in coords
sig_meta.ID = sig_meta.merge

In [29]:
from numpy import nan

def match_labels(row):
    curr_df = coords[(row['pos'] <= coords['end']) & (row['pos'] >= coords['start'])]
    try:
        row['ID'] = curr_df['ID'].iloc[0]
    except:
        row['ID'] = np.nan

    return row

result = sig_meta.apply(lambda x:match_labels(x),axis=1)

result

,MarkerName,A1,A2,Effect,StdErr,P-value,SNP,Z,pos,ID
0,chr16:31054607,t,c,-0.0831,0.0099,6.178000e-17,rs1108431,-8.393939,31054607,ENSG00000169851
1,chr10:121534732,t,c,0.0587,0.0095,5.506000e-10,rs196202,6.178947,121534732,ENSG00000074047
2,chr5:102446462,a,c,-0.0603,0.0103,4.716000e-09,rs257320,-5.854369,102446462,ENSG00000118733
3,chr16:31023273,a,g,-0.0852,0.0100,1.338000e-17,rs12447930,-8.520000,31023273,ENSG00000162949
4,chr4:951040,t,g,0.1074,0.0097,1.054000e-28,rs3733344,11.072165,951040,ENSG00000172554
...,...,...,...,...,...,...,...,...,...,...
1376,chr12:40727145,t,c,0.2018,0.0300,1.638000e-11,rs28365229,6.726667,40727145,ENSG00000084073
1377,chr1:205739396,t,g,-0.0862,0.0146,3.779000e-09,rs823139,-5.904110,205739396,ENSG00000117280
1378,chr4:90630901,t,g,-0.1540,0.0099,1.091000e-54,rs356174,-15.555556,90630901,ENSG00000058091
1379,chr6:32401261,a,g,-0.0773,0.0102,2.825000e-14,rs9268609,-7.578431,32401261,ENSG00000184007


In [30]:
#check how many id is missing
result.ID.isna().sum()
##total 1370 genes are significant in GWAS summary stats

11

In [31]:
#save result -- run only once
result.to_csv(r'./data_folder/gwas_id.csv',index=False)

In [33]:
#drop all columns except ID
gwas_genes = result.copy()
gwas_genes.drop(gwas_genes.iloc[:,0:9], axis=1, inplace=True)
gwas_genes.dropna(inplace=True)

gwas_genes

,ID
0,ENSG00000169851
1,ENSG00000074047
2,ENSG00000118733
3,ENSG00000162949
4,ENSG00000172554
...,...
1376,ENSG00000084073
1377,ENSG00000117280
1378,ENSG00000058091
1379,ENSG00000184007


In [34]:
#save result -- run only once
gwas_genes.to_csv(r'./data_folder/gwas_genes.csv',index=False)

compare these 1370 significant genes in GWAS summary stats with TWAS significant genes after TWAS pipeline is done.